## 평가 지표 정의 설명

본 프로젝트는 이진 분류(binary classification) 문제이며, 클래스 간 불균형(class imbalance)이 존재하므로 다음 지표를 사용한다.

1. **Accuracy**
   - 전체 문장 중 모델이 맞게 예측한 비율

2. **Precision (class-wise)**
   - 모델이 "핵심문장(1)"이라고 예측한 것 중 실제로 1인 비율

3. **Recall (class-wise)**
   - 실제 핵심문장(1) 중 1이라고 맞게 예측한 비율

4. **F1-score (class-wise)**
   - Precision과 Recall의 조화 평균

5. **Macro F1 (최종 평가 지표)**
   - 클래스별 F1-score 평균
   - 클래스 불균형 문제에서 더 적합

6. **Balanced Accuracy**
   - 클래스별 recall을 동일 가중 평균한 값
   - 클래스 불균형 시 필수

---

## 평가 프로토콜 설명

1. 전체 데이터(115,890 문장)를 weak_label 기준 Stratified Split  
   - Train: 80%  
   - Validation: 10%  
   - Test: 10%

2. Logistic Regression 모델을 학습  
   - TF-IDF 기반 피처 사용  
   - class_weight="balanced" 적용  
   - Validation 성능 기준에서 Logistic Regression이 가장 우수하여 최종 모델로 선택

3. Test set에서 최종 평가 수행  
   - 학습/검증에 사용되지 않은 데이터  
   - 모델의 일반화 성능 평가

4. 추가 평가  
   - 사람이 직접 라벨링한 Human Label 200개로 재평가  
   - Weak label과 Human label의 차이를 비교 분석

# 데이터, 모델 로드 및 준비

In [1]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, f1_score, classification_report, balanced_accuracy_score

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/ML_Project

test = pd.read_csv("weak_test.csv")

X_test = test["clean_sentence"].fillna("")
y_test = test["weak_label"]

print("Test 데이터 크기:", test.shape)
test.head()

# Logistic Regression 모델 로드
model = joblib.load("final_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

print("모델, 벡터 로드 완료")

Mounted at /content/drive
/content/drive/MyDrive/ML_Project
Test 데이터 크기: (11589, 5)
모델, 벡터 로드 완료


# 1. Weak Test

In [2]:
X_test_tfidf = tfidf.transform(X_test)

test_pred = model.predict(X_test_tfidf)

acc_test = accuracy_score(y_test, test_pred)
f1_test = f1_score(y_test, test_pred, average="macro")
bal_acc_test = balanced_accuracy_score(y_test, test_pred)

print("- Weak Test 성능 평가 -")
print("Accuracy:", acc_test)
print("Balanced Accuracy:", bal_acc_test)
print("Macro F1:", f1_test)

print("\nClassification Report:")
print(classification_report(y_test, test_pred))

- Weak Test 성능 평가 -
Accuracy: 0.8993873500733454
Balanced Accuracy: 0.7846360083856302
Macro F1: 0.620010904095031

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.91      0.95     11222
           1       0.19      0.66      0.29       367

    accuracy                           0.90     11589
   macro avg       0.59      0.78      0.62     11589
weighted avg       0.96      0.90      0.93     11589



# 2. 직접(human) 라벨링 기반 평가

In [3]:
human = pd.read_csv("human_label.csv")

print("Human Label 데이터 크기:", human.shape)
human.head()

import re
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

if "clean_sentence" not in human.columns:
    human["clean_sentence"] = human["sentence"].apply(preprocess)

X_human = human["clean_sentence"].fillna("")
y_human = human["label"]

X_human_tfidf = tfidf.transform(X_human)

human_pred = model.predict(X_human_tfidf)

acc_human = accuracy_score(y_human, human_pred)
f1_human = f1_score(y_human, human_pred, average="macro")
bal_acc_human = balanced_accuracy_score(y_human, human_pred)

print("- Human Label Test 성능 -")
print("Accuracy:", acc_human)
print("Balanced Accuracy:", bal_acc_human)
print("Macro F1:", f1_human)

print("\nClassification Report:")
print(classification_report(y_human, human_pred))

Human Label 데이터 크기: (200, 3)
- Human Label Test 성능 -
Accuracy: 0.69
Balanced Accuracy: 0.5558298731904388
Macro F1: 0.5244669427826354

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.96      0.81       133
           1       0.67      0.15      0.24        67

    accuracy                           0.69       200
   macro avg       0.68      0.56      0.52       200
weighted avg       0.68      0.69      0.62       200



# 최종 성능 비교

In [4]:
results = pd.DataFrame({
    "Dataset": ["Weak Test", "Human Test"],
    "Accuracy": [acc_test, acc_human],
    "Balanced Accuracy": [bal_acc_test, bal_acc_human],
    "Macro F1": [f1_test, f1_human]
})

results

,Dataset,Accuracy,Balanced Accuracy,Macro F1
0,Weak Test,0.899387,0.784636,0.620011
1,Human Test,0.690000,0.555830,0.524467
